<h1 style="color:#b57c01;">[Data Cleaning] 財政部出口統計</h1>

In [14]:
import pandas as pds
import os
import re
from dotenv import load_dotenv

csv_ouptut_file_name ="mof_export_statistic.csv"


load_dotenv()

data_path = os.environ.get("DOWNLOAD_DIR")
print ( f"data path: {data_path}")

csvfiles = os.listdir( data_path )
csvfiles_number = len ( csvfiles )
print ( f"files ( {csvfiles_number} ) : {csvfiles}" )




data path: /home/botsz/AIPE02/ESG_carbon_tax/tool_kit/mof_export_statistics/raw_data/
files ( 132 ) : ['mof_statistic_112_9.csv', 'mof_statistic_114_2.csv', 'mof_statistic_106_1.csv', 'mof_statistic_106_4.csv', 'mof_statistic_107_10.csv', 'mof_statistic_111_7.csv', 'mof_statistic_110_2.csv', 'mof_statistic_112_1.csv', 'mof_statistic_106_2.csv', 'mof_statistic_112_8.csv', 'mof_statistic_108_8.csv', 'mof_statistic_107_5.csv', 'mof_statistic_114_11.csv', 'mof_statistic_108_2.csv', 'mof_statistic_111_8.csv', 'mof_statistic_114_5.csv', 'mof_statistic_109_10.csv', 'mof_statistic_107_8.csv', 'mof_statistic_110_3.csv', 'mof_statistic_108_7.csv', 'mof_statistic_111_4.csv', 'mof_statistic_105_11.csv', 'mof_statistic_105_3.csv', 'mof_statistic_109_2.csv', 'mof_statistic_106_3.csv', 'mof_statistic_112_2.csv', 'mof_statistic_114_8.csv', 'mof_statistic_105_12.csv', 'mof_statistic_108_10.csv', 'mof_statistic_109_12.csv', 'mof_statistic_106_12.csv', 'mof_statistic_113_3.csv', 'mof_statistic_107_7.csv'

In [15]:
fPath = os.path.join ( data_path, csvfiles[0] ) 
df=pds.read_csv( fPath , encoding="big5" )
unique_values = df["國家/地區別"].unique()
unique_values

array(['<b>全球</b>', '亞洲', '中國大陸與香港', '中國大陸', '香港', '日本', '南韓',
       '新南向18國(註1)', '東協10國', '泰國', '馬來西亞', '印尼', '菲律賓', '新加坡', '越南',
       '汶萊', '緬甸', '柬埔寨', '寮國', '南亞', '印度', '巴基斯坦', '孟加拉', '斯里蘭卡', '尼泊爾',
       '不丹', '哈薩克', '中東', '伊朗', '伊拉克', '以色列', '科威特', '阿曼', '卡達', '沙烏地阿拉伯',
       '土耳其', '阿拉伯聯合大公國', '北美洲', '美國', '加拿大', '中美洲', '墨西哥', '南美洲', '巴西',
       '智利', '秘魯', '歐洲', '德國', '法國', '荷蘭', '英國', '俄羅斯', '奧地利', '比利時',
       '匈牙利', '義大利', '波蘭', '西班牙', '瑞典', '瑞士', '烏克蘭', '歐盟(註2)', '東歐', '非洲',
       '阿爾及利亞', '安哥拉', '埃及', '南非', '肯亞', '奈及利亞', '大洋洲', '澳大利亞', '巴布亞紐幾內亞',
       '紐西蘭', nan], dtype=object)

### Merge the CSV files ###

In [16]:
def parse_month( x ):
    
    if pds.isna ( x ):
        return pds.NA
    
    monthStr = str ( x )

    m_range = re.search ( r"\(?(\d+)\s*~\s*(\d+)月\)?", x )
    if m_range :
        return int ( m_range.group(1) ) + 100
    
    m_single = re.search(r"(\d+)月", x)
    if m_single:
        return int(m_single.group(1))
    
    return pds.NA


In [17]:

def data_cleaning ( dfs , csvfiles ):

    for fname in csvfiles :

        m = re.search(r"mof_statistic_(\d+)_(\d+)\.csv", fname)
        if not m:
            continue

        year  = int(m.group(1))
        month = int(m.group(2))

        fPath = os.path.join ( data_path, fname ) 
        tmp = pds.read_csv ( fPath , encoding="big5" )

        tmp[ "月" ] = tmp ["統計期"].apply ( parse_month ).astype( "Int64" )
        tmp = tmp.drop ( columns = [ "統計期" ] )

        tmp = tmp [ tmp [ "月" ] < 13 ]

        locationStrList = [ "<b>全球</b>" , "中國大陸與香港" ]
        for l in locationStrList :
            tmp = tmp [ tmp [ "國家/地區別" ] != l ]
        
        tmp = tmp [~tmp [ "國家/地區別" ].isna()]
        

        # add year and month
        tmp[ "年" ] = year
        tmp[ "年" ] = tmp [ "年" ].astype ( "Int64" )
        cols_old = list( tmp.columns )
        cols = [[ "年", "月" ] , cols_old ] 

        dfs.append( tmp )

dfs= []
data_cleaning ( dfs , csvfiles ) 
df = pds.concat ( dfs, ignore_index=True )
df = df.rename(columns={"<b>總計</b>": "總計"})





In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9504 entries, 0 to 9503
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   國家/地區別            9504 non-null   object
 1   總計                9504 non-null   object
 2   1.活動物；動物產品        9504 non-null   object
 3   (1)水產品            9504 non-null   object
 4   2.植物產品            9504 non-null   object
 5   (1)食用蔬菜           9504 non-null   object
 6   (2)食用果實及堅果        9504 non-null   object
 7   3.動植物油脂           9504 non-null   object
 8   4.調製食品；飲料及菸酒      9504 non-null   object
 9   (1)調製食品           9504 non-null   object
 10  (2)飲料、酒類及醋        9504 non-null   object
 11  (3)菸及菸葉代用品        9504 non-null   object
 12  5.礦產品             9504 non-null   object
 13  (1)石油煉製品          9504 non-null   object
 14  6.化學品             9504 non-null   object
 15  (1)有機化學品          9504 non-null   object
 16  7.塑膠、橡膠及其製品       9504 non-null   object
 17  (1)塑膠原料       

In [19]:
text_cols = [ "國家/地區別" ]
num_cols = df.columns.difference ( text_cols ) 

for l in num_cols :
    df[ l ] = pds.to_numeric ( df [ l ], errors="coerce").astype ( 'Int64' )
    df[ l ] = df[ l ].fillna(0).astype( 'Int64' )

df.head()

#df_selected = df[["統計期","年","月","國家/地區別", "總計"]].copy()
#df_selected



,國家/地區別,總計,1.活動物；動物產品,(1)水產品,2.植物產品,(1)食用蔬菜,(2)食用果實及堅果,3.動植物油脂,4.調製食品；飲料及菸酒,(1)調製食品,...,(4)資通與視聽產品,(5)家用電器,17.運輸工具,18.光學及精密儀器；鐘錶；樂器,(1)光學及精密儀器,19.其他,(1)家具,(2)玩具與運動用品,月,年
0,亞洲,25612272,108735,98451,31053,5738,2591,7045,133327,75936,...,2838529,12308,158577,652318,643735,189570,27415,55425,9,112
1,中國大陸,8134400,16518,12259,3093,47,60,257,35458,15059,...,1023007,3466,49443,273964,271683,44997,2500,16758,9,112
2,香港,5689049,10206,9918,4985,394,1692,1218,28473,22389,...,480524,900,3927,53382,50301,32845,1092,8973,9,112
3,日本,2191887,40536,36940,13142,5175,260,819,10960,5282,...,251533,3908,47733,89108,87233,48080,19284,14839,9,112
4,南韓,1432213,2270,2103,1005,1,118,2739,7417,5720,...,72297,803,9227,85280,84894,9470,1077,4027,9,112


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9504 entries, 0 to 9503
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   國家/地區別            9504 non-null   object
 1   總計                9504 non-null   Int64 
 2   1.活動物；動物產品        9504 non-null   Int64 
 3   (1)水產品            9504 non-null   Int64 
 4   2.植物產品            9504 non-null   Int64 
 5   (1)食用蔬菜           9504 non-null   Int64 
 6   (2)食用果實及堅果        9504 non-null   Int64 
 7   3.動植物油脂           9504 non-null   Int64 
 8   4.調製食品；飲料及菸酒      9504 non-null   Int64 
 9   (1)調製食品           9504 non-null   Int64 
 10  (2)飲料、酒類及醋        9504 non-null   Int64 
 11  (3)菸及菸葉代用品        9504 non-null   Int64 
 12  5.礦產品             9504 non-null   Int64 
 13  (1)石油煉製品          9504 non-null   Int64 
 14  6.化學品             9504 non-null   Int64 
 15  (1)有機化學品          9504 non-null   Int64 
 16  7.塑膠、橡膠及其製品       9504 non-null   Int64 
 17  (1)塑膠原料       

In [21]:
processed_data_path = os.environ.get("PROCESSED_DATA_DIR") 
csv_ouptut_file_name = os.path.join ( processed_data_path , csv_ouptut_file_name )
df.to_csv ( csv_ouptut_file_name, index=False, encoding="utf-8" )

df = pds.read_csv ( csv_ouptut_file_name )
df.head()

,國家/地區別,總計,1.活動物；動物產品,(1)水產品,2.植物產品,(1)食用蔬菜,(2)食用果實及堅果,3.動植物油脂,4.調製食品；飲料及菸酒,(1)調製食品,...,(4)資通與視聽產品,(5)家用電器,17.運輸工具,18.光學及精密儀器；鐘錶；樂器,(1)光學及精密儀器,19.其他,(1)家具,(2)玩具與運動用品,月,年
0,亞洲,25612272,108735,98451,31053,5738,2591,7045,133327,75936,...,2838529,12308,158577,652318,643735,189570,27415,55425,9,112
1,中國大陸,8134400,16518,12259,3093,47,60,257,35458,15059,...,1023007,3466,49443,273964,271683,44997,2500,16758,9,112
2,香港,5689049,10206,9918,4985,394,1692,1218,28473,22389,...,480524,900,3927,53382,50301,32845,1092,8973,9,112
3,日本,2191887,40536,36940,13142,5175,260,819,10960,5282,...,251533,3908,47733,89108,87233,48080,19284,14839,9,112
4,南韓,1432213,2270,2103,1005,1,118,2739,7417,5720,...,72297,803,9227,85280,84894,9470,1077,4027,9,112


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9504 entries, 0 to 9503
Data columns (total 45 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   國家/地區別            9504 non-null   object
 1   總計                9504 non-null   int64 
 2   1.活動物；動物產品        9504 non-null   int64 
 3   (1)水產品            9504 non-null   int64 
 4   2.植物產品            9504 non-null   int64 
 5   (1)食用蔬菜           9504 non-null   int64 
 6   (2)食用果實及堅果        9504 non-null   int64 
 7   3.動植物油脂           9504 non-null   int64 
 8   4.調製食品；飲料及菸酒      9504 non-null   int64 
 9   (1)調製食品           9504 non-null   int64 
 10  (2)飲料、酒類及醋        9504 non-null   int64 
 11  (3)菸及菸葉代用品        9504 non-null   int64 
 12  5.礦產品             9504 non-null   int64 
 13  (1)石油煉製品          9504 non-null   int64 
 14  6.化學品             9504 non-null   int64 
 15  (1)有機化學品          9504 non-null   int64 
 16  7.塑膠、橡膠及其製品       9504 non-null   int64 
 17  (1)塑膠原料       